# Model

In [1]:
import pandas as pd
from database import engine

from IPython.display import HTML

## Loading data

- We only load records that belong to the `view` category, because every like starts with view, and we have a `target` column that marks those views that lead to likes.

In [2]:
records_count = 1000

user_data = pd.read_sql(
    f"SELECT * FROM public.user_data LIMIT {records_count};",
    con = engine,
    index_col = "user_id"
)
post_data = pd.read_sql(
    f"SELECT * FROM public.post_text_df LIMIT {records_count};",
    con = engine,
    index_col = "post_id"
)
feed_data = pd.read_sql(
    f"SELECT * FROM public.feed_data limit {records_count};",
    con = engine
)

# joined data is where each user 
# is matched with all his actions
query = f"""
SELECT 
    public.feed_data.timestamp,
    public.feed_data.target,
    public.user_data.gender,
    public.user_data.age,
    public.user_data.country,
    public.user_data.city,
    public.user_data.exp_group,
    public.user_data.os,
    public.user_data.source,
    public.post_text_df.text,
    public.post_text_df.topic
FROM public.feed_data
LEFT JOIN public.user_data
    ON public.feed_data.user_id = public.user_data.user_id
LEFT JOIN public.post_text_df
    ON public.feed_data.post_id = public.post_text_df.post_id
WHERE
    public.feed_data.action='view'
LIMIT {records_count};
"""
joined_data = pd.read_sql(query, con = engine)

In [3]:
df_show = {
    "Users data" : user_data,
    "Post data" : post_data,
    "Feed data" : feed_data,
    "Joined data" : joined_data
}

for title, df in df_show.items():
    display(HTML(f"<h3>{title}</h3>"))
    display(df.head())

,gender,age,country,city,exp_group,os,source
user_id,,,,,,,
200,1,34,Russia,Degtyarsk,3,Android,ads
201,0,37,Russia,Abakan,0,Android,ads
202,1,17,Russia,Smolensk,4,Android,ads
203,0,18,Russia,Moscow,1,iOS,ads
204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads


,text,topic
post_id,,
1,UK economy facing major risks\n\nThe UK manufa...,business
2,Aids and climate top Davos agenda\n\nClimate c...,business
3,Asian quake hits European shares\n\nShares in ...,business
4,India power shares jump on debut\n\nShares in ...,business
5,Lacroix label bought by US firm\n\nLuxury good...,business


,timestamp,user_id,post_id,action,target
0,2021-10-24 17:49:07,162875,355,view,0
1,2021-10-24 17:51:10,162875,3293,view,0
2,2021-10-24 17:51:51,162875,3835,view,0
3,2021-10-24 17:52:03,162875,1254,view,0
4,2021-10-24 17:52:28,162875,806,view,0


,timestamp,target,gender,age,country,city,exp_group,os,source,text,topic
0,2021-10-24 17:49:07,0,0,36,Russia,Krasnoyarsk,3,iOS,organic,M&S cuts prices by average of 24%\n\nMarks & S...,business
1,2021-10-24 17:51:10,0,0,36,Russia,Krasnoyarsk,3,iOS,organic,We should not rush to start treating #COVID19 ...,covid
2,2021-10-24 17:51:51,0,0,36,Russia,Krasnoyarsk,3,iOS,organic,@davidpumpkins1 @PolitelyWest @wallisweaver Ri...,covid
3,2021-10-24 17:52:03,0,0,36,Russia,Krasnoyarsk,3,iOS,organic,Blair defends terror law changes\n\nThe prime ...,politics
4,2021-10-24 17:52:28,0,0,36,Russia,Krasnoyarsk,3,iOS,organic,OutKast win at MTV Europe Awards\n\nUS hip-hop...,entertainment


## Transformation

In [6]:
X = joined_data.copy()

y = X["target"]
X.drop("target", axis = 1, inplace = True)

X["month"] = X["timestamp"].dt.month
X["year"] = X["timestamp"].dt.year
X["hour"] = X["timestamp"].dt.hour
X.drop("timestamp", axis = 1, inplace = True)

## EDA

Actions types.